In [1]:
//q6 start
import java.io._
def localpath(path: String): String = {
    "file://" + new java.io.File(".").getCanonicalPath + "/" + path
}
//val reg = """\s+\<row\s+(\w+)=\"(.*?)\".*(\w+)=\"(\d+)\"""".r
val reg = """\s*<row\s*(\w+=".*?"\s*)*/>""".r
var valid_stack_posts_lines=sc.textFile(localpath("spark-stats-data/allPosts/"))
.filter(x=> reg.findFirstIn(x).isDefined)

//.filter(x => x.endsWith(" />") && x.startsWith("  <row"))

var valid_stack_users_lines = sc.textFile(localpath("spark-stats-data/allUsers/"))
.filter(x=> reg.findFirstIn(x).isDefined)
//.filter(x => x.endsWith(" />") && x.startsWith("  <row"))


Intitializing Scala interpreter ...

Spark Web UI available at http://jupyter-saman-2emonjezi-40gmail-2ecom:4043
SparkContext available as 'sc' (version = 2.3.1, master = local[*], app id = local-1550629179488)
SparkSession available as 'spark'


import java.io._
localpath: (path: String)String
reg: scala.util.matching.Regex = \s+\<row\s+(\w+)=\"(.*?)\".*(\w+)=\"(\d+)\"
valid_stack_posts_lines: org.apache.spark.rdd.RDD[String] = MapPartitionsRDD[2] at filter at <console>:33
valid_stack_users_lines: org.apache.spark.rdd.RDD[String] = MapPartitionsRDD[5] at filter at <console>:38


In [93]:
//exact numbers in notebook 4016004 4016017
valid_stack_posts_lines.filter(x => x.contains("PostTypeId=\"2\"")).count()
//valid_stack_posts_lines.filter(x => x.contains("PostTypeId=\"1\"")).count()

res57: Long = 55303


In [2]:
// NEW part 1
////q6 user creation date info new
import java.sql.Timestamp
import java.text.SimpleDateFormat

val dateFormat = new SimpleDateFormat("yyyy-MM-dd'T'HH:mm:ss.SSS") //hh for 24 format, Z for time zone such as -0800

def successMatches2(s: String): (Int,Long) = {
  val IdMatcher = """ Id=\"(\d+)\" """.r
  val CreationDateMatcher = """ CreationDate=\"(.*?)\" """.r
  val Id = IdMatcher.findFirstMatchIn(s).map(_.group(1)).getOrElse("0")
  val CreationDate = CreationDateMatcher.findFirstMatchIn(s).map(_.group(1)).getOrElse("0")
  val time = dateFormat.parse(CreationDate).getTime()
  (Id.toInt,time)
  }
//q6 dates to df
val q6_creationdate=valid_stack_users_lines.map(x => successMatches2(x))
//.map(line => (line._1,line._2,line._3)).sortBy(_._2, ascending=false).collect not this
.map(line => (line._1,line._2))
var cols2 : Array[String] = Array("Id", "cdate");
val df_q6_cdate= q6_creationdate.toDF(cols2: _*);
//var df_q6_cdate= valid_stack_users_lines.map(x => successMatches2(x))
//.map(line => (line._1,line._2,line._3)).collect.toSeq.toDF(cols2: _*)

import java.sql.Timestamp
import java.text.SimpleDateFormat
dateFormat: java.text.SimpleDateFormat = java.text.SimpleDateFormat@d2ec523f
successMatches2: (s: String)(Int, Long)
q6_creationdate: org.apache.spark.rdd.RDD[(Int, Long)] = MapPartitionsRDD[7] at map at <console>:50
cols2: Array[String] = Array(Id, cdate)
df_q6_cdate: org.apache.spark.sql.DataFrame = [Id: int, cdate: bigint]


In [3]:
//NEW part2
//q6 start, questions
import java.text.SimpleDateFormat
val dateFormat = new SimpleDateFormat("yyyy-MM-dd'T'HH:mm:ss.SSS")
def successMatches1(s: String): (String,Long,Int,Int,Int,Int) = {
  val OwnerUserIdMatcher = """ OwnerUserId=\"(\d+)\" """.r
  val CreationDateMatcher = """ CreationDate=\"(.*?)\" """.r
  val AnswerCountMatcher = """ AnswerCount=\"(.*?)\" """.r
  val ViewCountMatcher = """ ViewCount=\"(.*?)\" """.r
  val ScoreMatcher = """ Score=\"(\d+)\" """.r
  val FavoriteCountMatcher = """ FavoriteCount=\"(\d+)\" """.r
  val AnswerCount = AnswerCountMatcher.findFirstMatchIn(s).map(_.group(1)).getOrElse("0")
  val FavoriteCount = FavoriteCountMatcher.findFirstMatchIn(s).map(_.group(1)).getOrElse("0")
  val ViewCount = ViewCountMatcher.findFirstMatchIn(s).map(_.group(1)).getOrElse("0")
  val OwnerUserId = OwnerUserIdMatcher.findFirstMatchIn(s).map(_.group(1)).getOrElse("0")
  val CreationDate = CreationDateMatcher.findFirstMatchIn(s).map(_.group(1)).getOrElse("0")
  val Score = ScoreMatcher.findFirstMatchIn(s).map(_.group(1)).getOrElse("0")
  val time = dateFormat.parse(CreationDate).getTime()
  (OwnerUserId,time,Score.toInt,FavoriteCount.toInt,ViewCount.toInt,AnswerCount.toInt)
  }
val valid_stack_posts_lines_questions = valid_stack_posts_lines
.filter(x => x.contains(" PostTypeId=\"1\"") && x.contains("OwnerUserId=\""))

//all first questions
val q6_first_qs=valid_stack_posts_lines_questions.map(x => successMatches1(x))
.map(line => (line._1,(line._2,line._3,line._4,line._5,line._6))).reduceByKey {
  case ((dateL,favL,scL,vcL,acL), (dateR,favR,scR,vcR,acR)) => 
    (if (dateL > dateR) (dateR,favR,scR,vcR,acR) else (dateL,favL,scL,vcL,acL))
}.map(line => (line._1,line._2._1,line._2._2,line._2._3,line._2._4,line._2._5))
var cols1 : Array[String] = Array("Id", "cdate","Score","FavoriteCount","ViewCount","AnswerCount")
val df_q6_user_info= q6_first_qs.toDF(cols1: _*)
//var df_q6_user_info= valid_stack_posts_lines_questions.map(x => successMatches1(x))
//.map(line => (line._1,(line._2,line._3,line._4,line._5,line._6))).reduceByKey {
//  case ((dateL,favL,scL,vcL,acL), (dateR,favR,scR,vcR,acR)) => 
//    (if (dateL >= dateR) (dateR,favR,scR,vcR,acR) else (dateL,favL,scL,vcL,acL))
//}.map(line => (line._1,line._2._1,line._2._2,line._2._3,line._2._4,line._2._5))
//.toDF(cols1: _*);

import java.text.SimpleDateFormat
dateFormat: java.text.SimpleDateFormat = java.text.SimpleDateFormat@d2ec523f
successMatches1: (s: String)(String, Long, Int, Int, Int, Int)
valid_stack_posts_lines_questions: org.apache.spark.rdd.RDD[String] = MapPartitionsRDD[8] at filter at <console>:57
q6_first_qs: org.apache.spark.rdd.RDD[(String, Long, Int, Int, Int, Int)] = MapPartitionsRDD[12] at map at <console>:64
cols1: Array[String] = Array(Id, cdate, Score, FavoriteCount, ViewCount, AnswerCount)
df_q6_user_info: org.apache.spark.sql.DataFrame = [Id: string, cdate: bigint ... 4 more fields]


In [41]:
df_q6_user_info.filter(($"Id" >28)&&($"Id" <30)).show()

+---+-------------+-----+-------------+---------+-----------+
| Id|        cdate|Score|FavoriteCount|ViewCount|AnswerCount|
+---+-------------+-----+-------------+---------+-----------+
| 29|1288643477030|    8|            0|      440|          1|
+---+-------------+-----+-------------+---------+-----------+



In [6]:
//NEW part3
//q6 find vets
import java.text.SimpleDateFormat
val dateFormat = new SimpleDateFormat("yyyy-MM-dd'T'HH:mm:ss.SSS") 
val day = 24*60*60*1000L
def successMatches3(s: String): (String,Long) = {
  val OwnerUserIdMatcher = """ OwnerUserId=\"(\d+)\" """.r
  val CreationDateMatcher = """ CreationDate=\"(.*?)\" """.r
  val OwnerUserId = OwnerUserIdMatcher.findFirstMatchIn(s).map(_.group(1)).getOrElse("0")
  val CreationDate = CreationDateMatcher.findFirstMatchIn(s).map(_.group(1)).getOrElse("0")
  val time = dateFormat.parse(CreationDate).getTime()
  (OwnerUserId,time)
  }
val q6_postdate=valid_stack_posts_lines
.filter(x => x.contains("OwnerUserId=\"")).map(x => successMatches3(x))

val cols3 : Array[String] = Array("Id", "pcdate")
val df_q6_pc=q6_postdate.toDF(cols3: _*)
//var df_q6_pc=valid_stack_posts_lines.map(x => successMatches3(x)).collect.toSeq.toDF(cols3: _*);

//q6 join df 
val q6_vet_joined=df_q6_pc.join(df_q6_cdate, usingColumns=Seq("Id"), joinType="left")
//deleted .cast("int")
val df_q6_users=q6_vet_joined.withColumn("diff", ($"pcdate"-$"cdate")).na.drop()//.fill(0)?
//deleted >=
.filter(($"diff" > 100L*day) && ($"diff" < 150L*day))
//var df_q6_users=df_q6_pc.join(df_q6_cdate, usingColumns=Seq("Id"), joinType="left")
//.withColumn("diff", ($"pcdate"-$"cdate").cast("int")).na.drop()//.fill(0)?
//.filter(($"diff" >= 100) && ($"diff" < 150));
var df_q6_vets=df_q6_users.groupBy(col("Id")).agg(
      (min($"diff")/(100L*day)).cast("int").as(("vet")));
var q6_info=df_q6_user_info.join(df_q6_vets, usingColumns=Seq("Id"), joinType="left").na.fill(0)//.drop()
//--
//var q6_info=df_q6_user_info.join(valid_stack_posts_lines
//                                 .map(x => successMatches3(x)).collect.toSeq.toDF(cols3: _*)
//                                 .join(df_q6_cdate, usingColumns=Seq("Id"), joinType="left")
//.withColumn("diff", ($"pcdate"-$"cdate").cast("int")).na.drop()//.fill(0)?
//.filter(($"diff" >= 100) && ($"diff" < 150)).groupBy(col("Id")).agg(
//      (min($"diff")/100).cast("int").as(("vet"))), usingColumns=Seq("Id"), joinType="left").na.fill(0);

import java.text.SimpleDateFormat
dateFormat: java.text.SimpleDateFormat = java.text.SimpleDateFormat@d2ec523f
day: Long = 86400000
successMatches3: (s: String)(String, Long)
q6_postdate: org.apache.spark.rdd.RDD[(String, Long)] = MapPartitionsRDD[44] at map at <console>:72
cols3: Array[String] = Array(Id, pcdate)
df_q6_pc: org.apache.spark.sql.DataFrame = [Id: string, pcdate: bigint]
q6_vet_joined: org.apache.spark.sql.DataFrame = [Id: string, pcdate: bigint ... 1 more field]
df_q6_users: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [Id: string, pcdate: bigint ... 2 more fields]
df_q6_vets: org.apache.spark.sql.DataFrame = [Id: string, vet: int]
q6_info: org.apache.spark.sql.DataFrame = [Id: string, cdate: bigint ... 5 more fields]


In [10]:
valid_stack_posts_lines.count

res3: Long = 108874


In [7]:
//1782
q6_info.filter($"vet" === 1).count()

res1: Long = 1817


In [30]:
df_q6_vets.count()

res13: Long = 2027


In [21]:
//q6 reduce
val q6_get_average=q6_info.as[(String, Float,Integer,Integer,Integer,Integer,Integer)]
.rdd.map(x => (x._7,(x._3,x._4,x._5,x._6,1L)))
.reduceByKey {
  case ((scL,fcL,vcL,acL,countL), (scR,fcR,vcR,acR,countR)) => 
    (scL+scR,fcL+fcR,vcL+vcR,acL+acR,countL+countR)
}.mapValues { 
  case (sc,fc,vc,ac,count) => 
    (sc.toDouble/count.toDouble,fc.toDouble/count.toDouble,vc.toDouble/count.toDouble,ac.toDouble/count.toDouble,count)
}
.map(line => (line._1,line._2._1,line._2._2,line._2._3,line._2._4,line._2._5))

q6_get_average: org.apache.spark.rdd.RDD[(Integer, Double, Double, Double, Double, Long)] = MapPartitionsRDD[259] at map at <console>:77


In [22]:
q6_get_average.take(2)

res8: Array[(Integer, Double, Double, Double, Double, Long)] = Array((0,2.131222144030026,0.5792165141918837,556.6446633825944,0.9726014543748533,21315), (1,3.553905390539054,1.300880088008801,926.3982398239824,1.2981298129812981,1818))


In [6]:
q6_get_average.take(3)

res0: Array[(Integer, Double, Double, Double, Double)] = Array((0,1.3157641791746142,0.39184710364562086,1103.5978033573347,1.506742935421468), (1,2.359968561697668,0.8673157237744455,1844.0344896669696,1.8426197044183144))


In [7]:
//vet count
df_q6_vets.count()

res1: Long = 288285


In [14]:
//all count
q6_info.count()

res3: Long = 23133
